In [1]:
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

0.1.74


In [2]:
ws = Workspace.from_config()

Found the config file in: C:\Development\GitHub\marcelfranke\azureml\GetStarted\config.json


In [4]:
ws.get_details()

{'id': '/subscriptions/b208dd3b-2592-4e14-a626-cd6941369193/resourceGroups/Demos/providers/Microsoft.MachineLearningServices/workspaces/PlaygroundMLWorkspace',
 'name': 'PlaygroundMLWorkspace',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'description': '',
 'friendlyName': '',
 'containerRegistry': '/subscriptions/b208dd3b-2592-4e14-a626-cd6941369193/resourcegroups/demos/providers/microsoft.containerregistry/registries/playgroundmlwo9484445750',
 'keyVault': '/subscriptions/b208dd3b-2592-4e14-a626-cd6941369193/resourcegroups/demos/providers/microsoft.keyvault/vaults/playgroundmlwo8951480593',
 'applicationInsights': '/subscriptions/b208dd3b-2592-4e14-a626-cd6941369193/resourcegroups/demos/providers/microsoft.insights/components/playgroundmlwo6044255436',
 'identityPrincipalId': 'acde2ca7-e242-4737-a149-f2660a68eae0',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions

In [5]:
from azureml.core import Experiment

# create a new experiment
exp = Experiment(workspace=ws, name='BigDataLab')

In [10]:
import pyspark

def init():
    # read in the model file
    from pyspark.ml import Pipeline, PipelineModel
    from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, Bucketizer
    global pipeline

    pipeline = PipelineModel.load('pipelineModel')

def generate_api_schema():
    from azureml.api.schema.dataTypes import DataTypes
    from azureml.api.schema.sampleDefinition import SampleDefinition
    from azureml.api.realtime.services import generate_schema
    import os
    print("create schema")
    sample_input = "{\"OriginAirportCode\":\"SAT\",\"Month\":5,\"DayofMonth\":5,\"CRSDepHour\":13,\"DayOfWeek\":7,\"Carrier\":\"MQ\",\"DestAirportCode\":\"ORD\",\"WindSpeed\":9,\"SeaLevelPressure\":30.03,\"HourlyPrecip\":0}"
    inputs = {"input_df": SampleDefinition(DataTypes.STANDARD, sample_input)}
    os.makedirs('outputs', exist_ok=True)
    print(generate_schema(inputs=inputs, filepath="outputs/schema.json", run_func=run))

def run(input_df):
    from pyspark.context import SparkContext
    from pyspark.sql.session import SparkSession
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)
    response = ''

    try:
        import json
        # Set inferSchema=true to prevent the float values from being seen as strings
        # which can later cause the VectorAssembler to throw an error: 'Data type StringType is not supported.'
        df = spark.read.option("inferSchema", "true").json(sc.parallelize([input_df]))

        #Get prediction results for the dataframe
        score = pipeline.transform(df)
        predictions = score.collect()
        #response = df
        #Get each scored result
        for pred in predictions:
            confidence = str(pred['probability'][0]) if pred['prediction'] == 0 else str(pred['probability'][1])
            response += "{\"prediction\":" + str(pred['prediction']) + ",\"probability\":" +  confidence + "},"
        # Remove the last comma
        response = response[:-1]
    except Exception as e:
        return (str(e))

    # Return results
    return response

In [11]:
testInput = "{\"OriginAirportCode\":\"SAT\",\"Month\":5,\"DayofMonth\":5,\"CRSDepHour\":13,\"DayOfWeek\":7,\"Carrier\":\"MQ\",\"DestAirportCode\":\"ORD\",\"WindSpeed\":9,\"SeaLevelPressure\":30.03,\"HourlyPrecip\":0}"
testInput2 = "{\"OriginAirportCode\":\"ATL\",\"Month\":2,\"DayofMonth\":5,\"CRSDepHour\":8,\"DayOfWeek\":4,\"Carrier\":\"MQ\",\"DestAirportCode\":\"MCO\",\"WindSpeed\":3,\"SeaLevelPressure\":31.03,\"HourlyPrecip\":0}"
# test init() in local notebook# test  
init()

# test run() in local notebook
run("[" + testInput + "," + testInput2 + "]")

Exception: Java gateway process exited before sending its port number